In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
positions = {
    'QB': 0,
    'RB': 2,
    'WR': 4,
    'TE': 6,
    'DST': 16,
    'K': 17,
    'FLEX': 23
}
pages = {
    'QB': 3,
    'RB': 6,
    'WR': 8,
    'TE': 4,
    'DST': 1,
    'K': 2,
    'FLEX': 18,
}
weeks = range(1, 22)

In [ ]:
def cbs(pos, week):
    url = 'https://games.espn.com/ffl/leaders?&scoringPeriodId={week}&seasonId=2017&slotCategoryId={pos}&startIndex={i}'.format(pos=positions[pos], week=week, i="{i}")
    try:
        contents = requests.get(url.format(i=0)).content
        soup = BeautifulSoup(contents, "html.parser")
        rows = soup.find('table').find_all('tr')
        header = [td.text.split(',')[0].replace(u'\xa0', u'') for td in rows[2]]
        data = []
        for i in range(pages[pos]):
            contents = requests.get(url.format(i=50*i)).content
            soup = BeautifulSoup(contents, "html.parser")
            rows = soup.find('table').find_all('tr')
            data += [[td.text.split(',')[0].split('D/ST')[0] for td in row] for row in rows[3:]]
    except AttributeError:
        print("Failed: " + pos + " " + str(week))
        return
    filename = 'yahoo_{pos}_week{week}.csv'.format(pos=pos, week=week)
    with open(filename, 'w') as f:
        print('writing: ', filename)
        writer = csv.writer(f)
        try:
            writer.writerow(header)
            writer.writerows(data)
        except UnicodeEncodeError:
            print("Failed: " + pos + " " + str(week))
            print(header)
            print(data)
            return

In [ ]:
url = 'https://football.fantasysports.yahoo.com/f1/731507/players?status=A&pos=O&cut_type=9&stat1=S_PW_12&myteam=0&sort=AR&sdir=1&count=0'

In [ ]:
driver = webdriver.Firefox()
driver.get(url)

In [ ]:
elem = driver.find_element_by_id("login-username")
elem.clear()
elem.send_keys("weinky")
elem.send_keys(Keys.RETURN)

In [ ]:
# Manually enter the password in the password box and hit return
# or fill the password in for "" here and run this cell
elem = driver.find_element_by_id("login-passwd")
elem.clear()
elem.send_keys("")
elem.send_keys(Keys.RETURN)

In [ ]:
driver.get(url)

In [ ]:
elem = driver.find_element_by_class_name("Table")

In [ ]:
soup = BeautifulSoup(elem.get_attribute("innerHTML"), "html.parser")

In [ ]:
header = [th.text for th in soup.find('thead').find_all('tr')[1].find_all('th')]

In [ ]:
data = [[td.text for td in row.find_all('td')] for row in soup.find_all('tr') if len(row.find_all('td')) == len(header)]

In [ ]:
['FN', 'LN'] + header[3:-1]

In [ ]:
[row[1].split('\n')[2].split(' ')[:2] + row[3:-1] for row in data]

In [ ]:
os.chdir('yahoo')

In [ ]:
[cbs(pos, week) for pos in positions for week in weeks]

In [ ]:
os.chdir('..')